In [9]:
from typing import Dict
from rich import print
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#custom imports
from Load import Load_Type, Load_Category, Load, Wall, Concrete

In [10]:
#Soil Properties
gamma_s = 19.0 # kPa/m
ka = 0.33
ko = 0
kp = 3.0

# concrete properties
fc = 25 # MPa
Ec = 4500*(fc**0.5) # MPa
alpha1 = 0.91
beta1 = 0.81

C25 = Concrete('C25', fc)

In [11]:
# Geometry
# Wall
h = 1800 # mm
tw = 200 # mm
L = 1000 # mm

# Footing
tc = 300 # mm
toe = 250 # mm
b = 1500 # mm
D = 300 # mm
heel = b-toe-tw # mm

wall = Wall('wall1', h, tw, toe, b, D, heel, C25)

In [20]:
# Loads
# Soil
q_soil = ka*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa
q_eq = 0.25*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa

# compaction
q_comp = 5 # kPa

# surcharge
W_dl = 2.4 # kPa
W_ll = 4.8 # kpa

# Wall Loads
P_dl = 100 # kN/m
P_sl = 200 # kN/m

wall.Add_Load('SOIL', 'HYDROSTATIC', 'soil', q_soil)
wall.Add_Load('SIESMIC', 'HYDROSTATIC', 'eq', q_eq)

wall.Add_Load('COMPACTION', 'SURCHARGE', 'cop', q_comp)

wall.Add_Load('DEAD', 'SURCHARGE', 'dl', W_dl)
wall.Add_Load('LIVE', 'SURCHARGE', 'll', W_ll)

wall.Add_Load('DEAD', 'WALL', 'wall_dl', P_dl)
wall.Add_Load('SNOW', 'WALL', 'wall_sl', P_sl)

for k, load in wall.Loads.items():
    print(f"{k} - {load.Value}")

soil - 15.048

eq - 11.4

cop - 5

dl - 2.4

ll - 4.8

wall_dl - 100

wall_sl - 200

In [22]:
def create_load_combinations(loads_dict: Dict[str, Load]):
    # Define load factors and load combinations
    load_factors = {
        Load_Category.DEAD: 1.25,
        Load_Category.LIVE: 1.5,
    }

    # Aggregate values by Load_Type and Load_Category
    aggregated_values = {}
    for load in loads_dict.values():
        key = (load.Type, load.Category)
        if key in aggregated_values:
            aggregated_values[key] += load.Value
        else:
            aggregated_values[key] = load.Value

    # Define a helper function to compute combinations for a specific Load_Type
    def compute_combinations(load_type):
        dead_load = aggregated_values.get((load_type, Load_Category.DEAD), 0)
        live_load = aggregated_values.get((load_type, Load_Category.LIVE), 0)
        snow_load = aggregated_values.get((load_type, Load_Category.SNOW), 0)
        
        
        return {
            "1.4dl": 1.4 * dead_load,
            "1.25dl+1.5ll": 1.25 * dead_load + 1.5 * live_load
        }

    # Create combo dictionaries for each Load_Type
    combinations = {
        "hydro_combo_dict": compute_combinations(Load_Type.HYDOSTATIC),
        "surcharge_combo_dict": compute_combinations(Load_Type.SURCHARGE),
        "wall_combo_dict": compute_combinations(Load_Type.WALL),
    }

    return combinations

# Sample usage
combinations = create_load_combinations(wall.Loads)
print(combinations)

{
    'hydro_combo_dict': {'1.4dl': 0.0, '1.25dl+1.5ll': 0.0},
    'surcharge_combo_dict': {'1.4dl': 0.0, '1.25dl+1.5ll': 0.0},
    'wall_combo_dict': {'1.4dl': 0.0, '1.25dl+1.5ll': 0.0}
}

In [ ]:
# load cases
loads = {}
# Construction
q_soil
q_comp
blend = True
if blend:
    x = q_comp/(ka*gamma_s*(L/1000))*1000
    if x > (h+tc+D):
        x = 0
P_comp = q_comp*(h+tc+D)/1000*L/1000
P_soil = q_soil*x/2000

V = P_comp + P_soil
M = P_comp*(h+tc+D)/2000 + P_soil * (h+tc+D)/3000
geo_loads['LC1'] = [V,M]

# Service
q_soil
q_comp
blend = True
if blend:
    x = q_comp/(ka*gamma_s*(L/1000))*1000
    if x > (h+tc+D):
        x = 0
P_comp = q_comp*(h+tc+D)/1000*L/1000
P_soil = q_soil*x/2000

P = P_comp + P_soil
M = P_comp*(h+tc+D)/2000 + P_soil * (h+tc+D)/3000
loads['LC1'] = [P,M]